In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this(by clicking run or pressing Shift+Enter) will list all files under the input directory
import os
import torch
import torch.nn as N
from torch.optim import Adam
import torch.nn.functional as F
import torch.optim as op
# Any results you write to the current directory are saved as output.

In [5]:
dframe = pd.read_csv("../input/entity-annotated-corpus/ner.csv", encoding = "ISO-8859-1", error_bad_lines=False)

b'Skipping line 281837: expected 25 fields, saw 34\n'


In [6]:
dframe.head()



,Unnamed: 0,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,...,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
0,0,thousand,of,demonstr,NNS,lowercase,demonstrators,IN,lowercase,of,...,__start2__,__START2__,wildcard,__START2__,wildcard,__START1__,1.0,capitalized,Thousands,O
1,1,of,demonstr,have,VBP,lowercase,have,NNS,lowercase,demonstrators,...,__start1__,__START1__,wildcard,__START1__,capitalized,Thousands,1.0,lowercase,of,O
2,2,demonstr,have,march,VBN,lowercase,marched,VBP,lowercase,have,...,thousand,NNS,capitalized,Thousands,lowercase,of,1.0,lowercase,demonstrators,O
3,3,have,march,through,IN,lowercase,through,VBN,lowercase,marched,...,of,IN,lowercase,of,lowercase,demonstrators,1.0,lowercase,have,O
4,4,march,through,london,NNP,capitalized,London,IN,lowercase,through,...,demonstr,NNS,lowercase,demonstrators,lowercase,have,1.0,lowercase,marched,O


In [7]:
df1=dframe[['word','tag']]

In [8]:
data=[]
label=[]
a=[]
b=[]
for x in zip(df1.word,df1.tag):
    
    if x[0]!=".":
        a.append(x[0])
        b.append(x[1])
    
    else:
        data.append(a)
        label.append(b)
        a=[]
        b=[]
        

In [9]:
data[0]

['Thousands',
 'of',
 'demonstrators',
 'have',
 'marched',
 'through',
 'London',
 'to',
 'protest',
 'the',
 'war',
 'in',
 'Iraq',
 'and',
 'demand',
 'the',
 'withdrawal',
 'of',
 'British',
 'troops',
 'from',
 'that',
 'country']

In [10]:
label[0]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-geo',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-gpe',
 'O',
 'O',
 'O',
 'O']

In [11]:
data1=data[:20000]
label1=label[:20000]

In [12]:
maxlen=max(map(len,data1))

In [13]:
maxlen

70

In [14]:
from collections import Counter,defaultdict

In [15]:
words=set()
for i in data:
    words.update(i)

In [16]:
word2idx=dict([(j,i+1) for i,j in enumerate(words)])

In [17]:
idx2word=dict([(j,i) for i,j in word2idx.items()])

In [18]:
idxdata=[[word2idx[j] for j in s] for s in data]

In [19]:
label1=set()
for i in label:
    label1.update(i)

In [20]:
word2idx['<pad>']=0
idx2word[0]='<pad>'

In [21]:
label2idx=dict([(i,j+1) for j,i in enumerate(label1)])

In [22]:
idx2label=dict([(i,j) for j,i in label2idx.items()])

In [23]:
idxlabel=[[label2idx[k] for k in s] for s in label]

In [24]:
idxlabel[:3]

[[5, 5, 5, 5, 5, 5, 6, 5, 5, 5, 5, 5, 6, 5, 5, 5, 5, 5, 12, 5, 5, 5, 5],
 [5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  1,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5,
  5],
 [5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 14]]

In [25]:
idxdata[:3]

[[2478,
  24450,
  11653,
  25141,
  6492,
  9527,
  20901,
  23014,
  2213,
  25430,
  21465,
  12412,
  22878,
  13172,
  20002,
  25430,
  2902,
  24450,
  20243,
  26732,
  9358,
  21285,
  27260],
 [18815,
  24450,
  3517,
  18104,
  12412,
  25430,
  4697,
  6012,
  25430,
  29767,
  1420,
  11930,
  16836,
  9522,
  22384,
  26919,
  29240,
  5226,
  3416,
  8968,
  8504,
  5342,
  5226,
  13172,
  5226,
  12899,
  25430,
  6746],
 [5226,
  3959,
  6492,
  9358,
  25430,
  25454,
  24450,
  18280,
  23014,
  29583,
  18335,
  12412,
  14782,
  6333]]

In [26]:
max(map(len,data))

96

In [27]:
def generator(data,label,batchsize,nepochs):
    maxlen=max(map(len,data))
    nbatches=len(data)/batchsize
    data=data[:int(nbatches*batchsize)]
    label=label[:int(nbatches*batchsize)]
    for i in range(nepochs):
        idx=np.random.permutation(np.arange(nbatches*batchsize).astype(np.int))
        data=data[idx]
        label=label[idx]
        
        for j in range(int(nbatches)):
            batchx=np.zeros((batchsize,maxlen))
            batchy=np.zeros((batchsize,maxlen))
            x=data[j*batchsize:(j+1)*batchsize]
            y=label[j*batchsize:(j+1)*batchsize]
            for m in range(batchsize):
                batchx[m,:len(x[m])]=x[m]
                batchy[m,:len(y[m])]=y[m]
            
            yield batchx,batchy

In [47]:
len(data)/100

47895

In [28]:
idxdata=np.array(idxdata)

In [29]:
idxlabel=np.array(idxlabel)

In [30]:
class model(N.Module):
    def __init__(self,vocabsize,embeddingdim,hiddendim,outputdim):
        super().__init__()
        self.embedding=N.Embedding(vocabsize,embeddingdim)
        self.lstm=N.LSTM(embeddingdim,hiddendim)
        self.fc=N.Linear(hiddendim,outputdim)
        
    def forward(self,x):
        embeds=self.embedding(x)
        #print(embeds.shape)
        embeds=embeds.permute(1,0,2)
        out1=self.lstm(embeds)   # seqlen x batchsize x 
        #print(out1[0].shape)
        out=self.fc(out1[0].permute(1,0,2))
        #print(out.shape)
        return out
        
    

In [31]:
label2idx['<pad>']=0

In [32]:
idx2label[0]='<pad>'

In [33]:
vocabsize=len(idx2word)
embeddim=164
hiddendim=256
outputdim=len(label2idx)

In [34]:
model1=model(vocabsize,embeddim,hiddendim,outputdim)

In [35]:
model1

model(
  (embedding): Embedding(30173, 164)
  (lstm): LSTM(164, 256)
  (fc): Linear(in_features=256, out_features=19, bias=True)
)

In [36]:
def count(model1):
    return sum([l.numel() for l in model1.parameters()])

In [37]:
count(model1)

5385383

In [38]:
optimizer=op.Adam(model1.parameters(),lr=0.0001)
loss=N.CrossEntropyLoss()

In [39]:
nepochs=2
batchsize=100

In [40]:
gen1=generator(idxdata,idxlabel,batchsize,nepochs)

In [41]:
device=torch.device("cuda" if torch.cuda.is_available() else 'cpu')

In [42]:
device

device(type='cuda')

In [44]:
model1=model1.to(device)
loss=loss.to(device)

In [46]:
trainloss=[]
count=0
for i in range(nepochs):
    for j in range(int(len(idxdata)/batchsize)):
        batchx,batchy=next(gen1)
       
        batchx1=torch.from_numpy(batchx).long()
        batchy1=torch.from_numpy(batchy).long()
        #print(batchx1)
        optimizer.zero_grad()
        batchx1=batchx1.to(device)  # very important to move tensors on gpu on which computations are taking place
        batchy1=batchy1.to(device)  # we moved the model and all tensors for computation on gpu
        output=model1.forward(batchx1)
        
        print(output.shape)
        print(batchy1.shape)
        l=loss(output.view(-1,19),batchy1.view(-1,)) #this is really important actual values should be of  shape (N), while pred (N,c)
        l.backward()
        optimizer.step()
        print("loss is:-",l," and epoch is:-",i," count is ",j)
    

torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(2.9288, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  0
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(2.8619, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  1
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(2.7926, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  2
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(2.7295, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  3
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(2.6710, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  4
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(2.6024, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  5
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(2.5397, device='cuda:0', grad_fn=<NllLossBackwa

loss is:- tensor(0.6828, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  62
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.7025, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  63
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.6784, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  64
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.7042, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  65
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.6567, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  66
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.6937, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  67
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.6488, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  68
torch

loss is:- tensor(0.5984, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  117
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.5990, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  118
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.5737, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  119
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.5992, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  120
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.5767, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  121
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.5885, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  122
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.5838, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  12

loss is:- tensor(0.3423, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  172
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.3700, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  173
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.3408, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  174
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.3437, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  175
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.3430, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  176
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.3503, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  177
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.3418, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  17

loss is:- tensor(0.2806, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  228
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.2692, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  229
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.2887, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  230
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.2711, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  231
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.2794, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  232
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.2621, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  233
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.2813, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 0  count is  23

...

loss is:- tensor(0.1651, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  273
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.1584, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  274
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.1839, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  275
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.1717, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  276
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.1614, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  277
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.1832, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  278
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.1570, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  27

loss is:- tensor(0.1691, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  328
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.1517, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  329
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.1677, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  330
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.1589, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  331
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.1764, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  332
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.1814, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  333
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.1628, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  33

torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.1761, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  384
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.1719, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  385
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.1565, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  386
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.1559, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  387
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.1856, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  388
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.1726, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  389
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.1469, device='cuda:0', grad_fn=<N

loss is:- tensor(0.1596, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  439
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.1573, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  440
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.1610, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  441
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.1640, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  442
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.1554, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  443
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.1652, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  444
torch.Size([100, 96, 19])
torch.Size([100, 96])
loss is:- tensor(0.1770, device='cuda:0', grad_fn=<NllLossBackward>)  and epoch is:- 1  count is  44

StopIteration: 